In [26]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime
import os


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [59]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

In [28]:


# Función para normalizar texto
def normalize_text(text):
    # Reemplazar caracteres especiales y convertir a minúsculas para una comparación consistente
    text = text.replace("/", " ")  # Reemplazar la barra por espacio o eliminarla completamente
    text = text.replace("ñ", "n")  # Reemplazar la "ñ" por "n"
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [29]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
    
    return df


In [30]:
def porcentaje_nulos(df, columnas):
    if isinstance(columnas, str):
        columnas = [columnas]  # Convertir a lista si es una sola columna
    resultados = {}
    for columna in columnas:
        if columna in df.columns:
            total_filas = len(df)
            nulos = df[columna].isnull().sum()
            porcentaje_nulos = (nulos / total_filas) * 100
            resultados[columna] = porcentaje_nulos
        else:
            resultados[columna] = 'Column not found'
    return resultados



# Carga de datos

In [31]:

print(os.getcwd())

c:\Users\extas\Documents\GitHub\precios-de-vivienda-madrid\codigo\01-preprocesamiento-de-datos\Revision_columnas


In [32]:
df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_14888\2425994981.py:1: DtypeWarning: Columns (3,5,14,26,30,68,69,74,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)


In [33]:
df17['cod_barrio'].value_counts(dropna=False)

cod_barrio
42.00     461
15.00     412
NaN       386
41.00     345
46.00     334
         ... 
193.00      3
4.00        3
7.00        1
88.00       1
1.00        1
Name: count, Length: 137, dtype: int64

In [34]:
df17 = df17.loc[:, ~df17.columns.str.contains('^Unnamed')]

In [35]:
df17.shape

(10430, 110)

In [36]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

# Revision primeras 31 columnas Gilberto

In [37]:
# Selecciono las primeras 31 columnas. De la 0 a la 30.
df_131=df17.iloc[:, :31]

In [38]:
# Checo la forma de mi dataframe y que realmente tome 31 columnas.
df_131.shape
# Muestreo de las primeras 5 filas 
df_131.head()
#Muestreo de las ultimas 5 filas 
df_131.tail()
# Muestreo de 5 filas aleatorias 
df_131.sample(5)


,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria
9098,Piso en venta en SanchinarroValdefuentes Hortaleza Madrid Madrid,https://www.redpiso.es/inmueble/piso-en-venta-en-sanchinarro-valdefuentes-hortaleza-madrid-madrid-RP1122023119185,355000.0,NaN,NaN,NaN,65.00,Hortaleza,16.00,Piovera,162.00,SANCHINARRO,NaN,NaN,NaN,NaN,NaN,Piso,NaN,Estado Perfecto Estado,0,NaN,NaN,2.00,1.0,NaN,NaN,Garajes 1,0.00,1,NaN
686,"Casa unifamiliar 5 habitaciones, Aravaca, Madrid",https://www.indomio.es/anuncios/95807161/,3300000.00,NaN,NaN,4.12,800.00,Moncloa-Aravaca,9.00,Aravaca,97.00,aravaca,NaN,Casa unifamiliar 5 habitaciones,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,NaN,5,5.00,3+,NaN,1.00,NaN,NaN,1.00,NaN
4789,Piso en venta en Calle de Ponce de León,https://www.pisos.com/comprar/piso-chamberi_almagro28010-45091025127_109300/,970000.00,NaN,NaN,7348.00,132.00,Chamberí,7.00,Almagro,74.00,NaN,NaN,NaN,NaN,NaN,NaN,Piso,NaN,NaN,NaN,NaN,NaN,2.00,3.0,0.00,0.00,NaN,0.00,0.00,NaN
7813,Piso en venta en CALLE CASTILLO DE SIMANCAS Simancas San BlasCanillejas Madrid Madrid,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-castillo-de-simancas-simancas-san-blas-canillejas-madrid-madrid-RP952024129554,165000.00,NaN,NaN,NaN,50.00,San Blas-Canillejas,20.00,Simancas,201.00,SIMANCAS,NaN,NaN,NaN,NaN,NaN,Piso,NaN,Estado Perfecto Estado,0,NaN,NaN,2.00,1.0,NaN,NaN,NaN,0.00,0.00,NaN
169,"Piso de tres habitaciones muy buen estado, octava planta, San Pascual, Madrid",https://www.indomio.es/anuncios/93521293/,289000.00,NaN,NaN,2.92,99.00,Ciudad Lineal,15.00,SanPascual,155.00,San Pascual,NaN,"Piso de tres habitaciones muy buen estado, octava planta",desconocido,NaN,No indicado,Piso,NaN,Óptimo / Reformado,NaN,8,3,3.00,2,NaN,1.00,NaN,1.00,1.00,NaN


In [39]:
# Chqueo de nulos en todas las columnas

df_131.isna().sum()
# Muestreo estadistico de las columnas

title                     0
url                       0
precio                    0
precio_anterior       10376
descuento             10376
EUR/m2                 2379
m2_constr                28
distrito                 22
cod_distrito             69
barrio                  411
cod_barrio              386
zona                   4805
calle                  9368
detalle                4506
consumoce_ano          7089
letrace                9039
emisiones_co2          6828
tipologia                 0
propiedad_completa    10319
estado                 4761
amueblado              7533
planta                 4645
habitaciones           7183
dormitorios             191
banos                   305
balcon                 4483
terraza                4359
garaje                 8374
trastero               1700
ascensor               2311
porteria              10278
dtype: int64

In [40]:

df_131.describe().T

,count,mean,std,min,25%,50%,75%,max
precio_anterior,54.00,812929.63,977732.38,98100.00,197750.00,432000.00,949875.00,4900000.00
EUR/m2,8051.00,3628.73,4001.52,0.00,6.58,2846.00,6246.50,27708.00
m2_constr,10402.00,248.27,6465.93,0.00,67.00,97.00,150.00,656600.00
cod_distrito,10361.00,7.71,5.75,0.00,4.00,6.00,12.00,21.00
cod_barrio,10044.00,81.49,57.31,1.00,41.00,65.00,124.00,215.00
propiedad_completa,111.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00
dormitorios,10239.00,2.81,1.43,0.00,2.00,3.00,3.00,24.00
terraza,6071.00,0.32,0.47,0.00,0.00,0.00,1.00,1.00
trastero,8730.00,0.16,0.36,0.00,0.00,0.00,0.00,1.00


In [41]:
df_131.describe().T
# Muestreo de tipos de columnas y nulos 
df_131.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10430 entries, 0 to 10429
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               10430 non-null  object 
 1   url                 10430 non-null  object 
 2   precio              10430 non-null  object 
 3   precio_anterior     54 non-null     float64
 4   descuento           54 non-null     object 
 5   EUR/m2              8051 non-null   float64
 6   m2_constr           10402 non-null  float64
 7   distrito            10408 non-null  object 
 8   cod_distrito        10361 non-null  float64
 9   barrio              10019 non-null  object 
 10  cod_barrio          10044 non-null  float64
 11  zona                5625 non-null   object 
 12  calle               1062 non-null   object 
 13  detalle             5924 non-null   object 
 14  consumoce_ano       3341 non-null   object 
 15  letrace             1391 non-null   object 
 16  emis

In [42]:
# Chequeo de nulos en la columna title 
df_131["title"].isna().sum()
# Filtro para checar si las filas que no tienen valores en title tienen valores en otras columnas importantes
#filas_con_nulos= df_131[df_131["title"].isnull()]
#ilas_con_nulos

0

In [43]:
df_131["title"].head(20)
df_131["title"].sample(20)

4063                                        Piso en venta en Calle de San Raimundo, cerca de Calle de Tenerife
2407                                                Piso de una habitación muy buen estado, San Andrés, Madrid
6150     Piso en venta en Avenida del Monasterio de El Escorial, cerca de Calle del Monasterio de Montesclaros
4143                                                                                 Piso en venta en Comillas
9818                                                                         Piso en venta en Recoletos Madrid
4223                                                               Piso en venta en Calle de San Sinforiano, 4
6280                                                                  Piso en venta en Calle del Doctor Espina
7486                                                                         Piso en venta en Calle de la Luna
7530                                                                 Piso en venta en Calle del Duque de Osuna
9

In [44]:
df_131["url"].sample(10)
df_131["url"].isna().sum()

0

In [45]:
# Filtro de filas con valores pequeños que no podrian ser el coste de un inmueble
df_131[df_131["precio"] == 1]

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria
758,"Piso de una habitación planta baja, Sol, Madrid",https://www.indomio.es/anuncios/95899949/,1.00,NaN,NaN,NaN,NaN,Centro,1.00,Sol,16.00,Sol,NaN,Piso de una habitación planta baja,desconocido,NaN,No indicado,Piso,NaN,NaN,NaN,Planta baja,1,1.00,1,0.00,NaN,NaN,1.00,1.00,NaN


se revisa la url https://www.indomio.es/anuncios/95899949/ y en l web aparece con valor 1€, lo consideramos un error, elimino la linea del dataset

In [46]:
df17.shape

(10430, 110)

In [47]:
# Eliminar filas donde la columna 'precio' es igual a 1
df17 = df17[df17["precio"] != 1]


In [48]:
df17.shape

(10429, 110)

In [ ]:

# Filtro de las filas con valores en precio anteior 
filas_sin_nulos= df_131[~df_131["precio_anterior"].isnull()]
filas_sin_nulos.head(4)
# Aqui filtre los links de los inmuebles con el precio anterior para checar algunos directamente y 
# ver si habia algo relevante del porque del cambio del precio.
filas_sin_nulos["url"]
# Chequeo de porcentaje de nulos en la columna precio_anterior
df_131["precio_anterior"].isna().mean()*100
df_131.shape
# Porcentaje de nulos 
df_131["descuento"].isna().mean()*100
#Porcentaje de nulos
df_131["EUR/m2"].isna().mean()*100
# Chequeo de nulos totales
df_131["EUR/m2"].isna().sum()
df_131["EUR/m2"].describe()
# Filtro de valores iguales a 0
filas_valor_eur= df_131[df_131["EUR/m2"] == 0]
filas_valor_eur

In [ ]:
# Filtro de valores 
filas_valor_eur_pequeños= df_131[df_131["EUR/m2"] <=100 ]
filas_valor_eur_pequeños.tail(5)
# Filtro para valores menores a 99 con un decimal. de esa forma los valores podran ser filtrados para no tener un precio mayor a 4 cifras por metro cuadrado
# que es lo que se ha notado donde predominan los valores.  Aqui cheque los valores con Data Wrangler y pude visualizar que los valores no son correctos. A algunos 
# les sirve solo colocarle 000 ceros extras al valor por metro cuadrado pero a otros no les sirve eso. 

filtered_df_131 = df_131[df_131['EUR/m2'].apply(lambda x: x < 99 and (x * 10).is_integer())]
filtered_df_131.head(5)
# Filtro por valores que parecen atipicos e irregulares para identificar vericidad
filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
filas_valor_eur2.head()
# Filtro del link por valores que parecen atipicos e irregulares para identificar vericidad
filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
filas_valor_eur2["url"]
# Porcentaje de nulos por columna
df_131.isna().mean()*100

In [ ]:
df17.query("cod_barrio.isnull() & cod_distrito.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','zona','calle','descripcion_extendida']]

## m2_constr

In [51]:
# Analisis descriptivo de la columna m2/constr 
df17["m2_constr"].describe()

count    10402.00
mean       248.27
std       6465.93
min          0.00
25%         67.00
50%         97.00
75%        150.00
max     656600.00
Name: m2_constr, dtype: float64

In [55]:

#Filtro de valores menores a 20 metros cuadrados. Se analizo con 10 tambien y siguen siendo mas de 200
df17[df17["m2_constr"] <= 10]['web'].unique()

array(['pisos.com', 'pisosmadrid'], dtype=object)

In [61]:
df17.query("m2_constr<= 10 & web=='pisosmadrid'")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina
10172,Piso en venta en Pueblo Nuevo Madrid,https://www.pisosmadrid.com.es/propiedad/4056165-piso-en-venta-en-pueblo-nuevo-madrid,212500,NaN,NaN,NaN,0.00,Ciudad Lineal,15.00,PuebloNuevo,152.00,Pueblo Nuevo,NaN,NaN,NaN,NaN,NaN,Piso,NaN,NaN,0.0,NaN,NaN,2.00,1,NaN,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,Piso 2 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid Ubicado en el barrio de Pueblo Nuevo Madrid Piso exterior que consta de 2 habitaciones 1 bano saloncomedor cocina y terraza La vivienda esta,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,4056165,10.00,2024-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,28017.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00,sin_informacion
10181,Apartamento en venta en Ventas Madrid,https://www.pisosmadrid.com.es/propiedad/4056185-apartamento-en-venta-en-ventas-madrid,345200,NaN,NaN,NaN,0.00,Ciudad Lineal,15.00,Ventas,151.00,ventas,NaN,NaN,NaN,NaN,NaN,Apartamento,NaN,NaN,0.0,NaN,NaN,3.00,2,NaN,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,Apartamento 3 dormitorios 2 banos 0 garajes Para reformar en Madrid Madrid Ubicado en el Barrio de Ventas Madrid Piso recien reformado distribuido en saloncomedor con cocina abierta 3 habitaciones 2 banos y 2 t,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,4056185,30.00,2024-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,28027.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.00,sin_informacion


# En el caso de pisos madrid se visita la web, no existe campo con el valor pero lo poner en la descripcion, se asigna el valor de 2 de los  pisos

In [60]:
# Asignar valor específico si la URL coincide
df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/3912876-piso-en-venta-en-malasana-madrid", "m2_constr"] = 131

df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/4056174-apartamento-en-venta-en-centro-madrid", "m2_constr"] = 100



In [63]:
df17.query("m2_constr<= 10 & web=='pisos.com'")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina
3111,"Apartamento en venta en Calle de los Misterios, 16, cerca de Calle de la Virgen de Lluc",https://www.pisos.com/comprar/apartamento-ciudad_lineal_quintana-48353042971_100500/,220000.00,NaN,NaN,6875.00,2.00,Ciudad Lineal,15.00,Quintana,153.00,NaN,NaN,"Si estás buscando una vivienda para independizarte o para invertir , ¡Deja de buscar, esta es tu oportunidad!\n\nAsr home te presenta genial apartamento, con garaje y trastero en una de las mejores zonas de madrid. \n\nEste piso es especial porque: por la luz que tiene, la ubicación y por tener todo lo que necesitas.\n\nDistribucion: al ser un apartamento , todo esta a la mano, la cocina y salón todo junto y el baño independiente.\n\nEquipamiento y calidades: el piso se encuentra para entrar a vivir con calidades de construcción muy buenas y en condiciones muy aceptables.\n\nComunicaciones: se ubica cerca del metro ciudad lineal y múltiples paradas de autobuses que le llevaran a cualquier lugar del centro de madrid. \n\nComercio, servicios, ocio: en esta zona hay gran variedad de comercio, restauración, centros médicos y hospitales, \n\nCentros educativos: en el sector de la enseñanza la zona cuenta con una estupendas guarderías y escuela de primaria. Para los otros niveles hay colegios públicos y privados.\n\nIbi 414.39 €/año\nComunidad 114,17 €/mes\n\nTe financiamos hasta el 100% no dudes en consultarno.",NaN,NaN,NaN,Apartamento,NaN,NaN,NaN,6875.0,NaN,1.00,32.0,0.00,0.00,NaN,1.00,1.00,NaN,Gas natural,Entre 10 y 20 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pisos.com,2024-10-19,Disponible,NaN,NaN,NaN,153.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion
3141,"Apartamento en venta en Calle del Teniente Coronel Noreña, 22",https://www.pisos.com/comprar/apartamento-legazpi_barrio28045-29263829710_109700/,410415.00,NaN,NaN,6728.00,1.00,Arganzuela,2.00,Legazpi,24.00,NaN,NaN,Apartamento en un edificio de obra nueva de 61 m2 de superficie construida y ubicado en la zona de Madrid Capital. Está distribuido en 1 habitación.,NaN,NaN,NaN,Apartamento,NaN,A estrenar,NaN,61.0,NaN,1.00,1.0,0.00,0.00,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

En pisos.com hay 198 filas con m2_contr erronea, se va al origen para ver que transformación se realizó

In [64]:
#utilizo el fichero pisoscom_limpio que parece que aun tenia los valores de m2_constr
dfpisoscomlimpio= pd.read_csv('./02-datos-limpios/pisoscom_limpio.csv',index_col=False)

FileNotFoundError: [Errno 2] No such file or directory: './02-datos-limpios/pisoscom_limpio.csv'

In [ ]:

filas_valor_m2.shape
# filtro por columnas url 
filas_valor_m2["url"]
# Muestreo de dos columnas especificas
filas_valor_m2[["url", "m2_constr"]]
#Muestreo/Filtro de las filas sin valores nulos 
m2_constr_filtro= df_131[~df_131["m2_constr"].isna()]
m2_constr_filtro.sample(5)           

In [ ]:
                                                                                                               
# Alcance a terminar m2_constr y me quede en distrito.
df_131["distrito"].unique()
# Muestreo de valores unicos 
df_131["cod_distrito"].unique()
# Muestreo de valores estadisticos 
df_131["cod_distrito"].describe()
# filtro para encontrar los valores iguales a 0 
cero_distrito= df_131[df_131["cod_distrito"]== 0]
cero_distrito
# Muestreo de valores unicos 
df_131["barrio"].unique()
# Muestreo de valores estadisticos
df_131["cod_barrio"].describe()
# Filtro de valores igual a 0
cero_barrio= df_131[df_131["cod_barrio"]== 0]
cero_barrio
# Muestreo de valores unicos 
df_131["zona"].unique()
# Muestreo de valores unicos 
df_131["calle"].unique()
# Muestreo de valores aleatorios 
df_131["detalle"].sample(15)
# Muestreo de valores aleatorios
df_131["detalle"].sample(15)
# Muestreo de valores totales por cada valor unico
df_131["consumoce_ano"].value_counts()
#  Muestreo de nulos totales 
df_131["consumoce_ano"].isna().sum()
# Muestreo de los valores unicos 
df_131 ["letrace"].unique()
# Muestreo de los valores unicos
df_131["emisiones_co2"].unique()
# Muestreo aleatorios de valores en dos columnas 
df_131[["letrace","emisiones_co2"]].sample(10)
#  Muestreo de valores unicos 
df_131["tipologia"].unique()
# Muestreo de valores unicos
df_131["propiedad_completa"].unique()
# Muestreo de valores unicos 
df_131["estado"].unique()
# Muestreo de valores unicos 
df_131["amueblado"].unique()
#  Seleccion de columnas y filtro por indice
filtro_planta= df_131[["planta","url"]]
filtro_planta["url"][7762]
# Muestreo de valores aleatorios
df_131["planta"].sample(10)
#  Muestreo de valores unicos 
df_131["habitaciones"].unique()
#  Muestreo de valores unicos
df_131["dormitorios"].unique()
#  Filtro de valores mayor o igual a 20  
dormitorios_filtro=df_131[df_131["dormitorios"]>=20]
dormitorios_filtro["url"]
#  Muestreo de valores unicos 
df_131["banos"].unique()
#  Filtro de valores igual a 397.0 ....Aqui se puede ver claramente que en la descripcion menciona 3 baños y en la columna de banos aparecen 397.0.
banos_filtro= df_131[df_131["banos"]=="397.0"] 
banos_filtro
# filtro por url
banos_filtro["url"]
#  Filtro directo al valor "278.0" . Entrando al link se puede apreciar que el valor en si mismo no hace referencia a la cantidad de banos sino al espacio
# total del inmueble o llamado el pagina como superficie construida.
banos_filtro= df_131[df_131["banos"]=="278.0"] 
banos_filtro
# Muestreo de valores unicos 
df_131["balcon"].unique()
# Muestreo de valores unicos
df_131["terraza"].unique()
# Muestreo de valores unicos
df_131["garaje"].unique()
# Muestreo de valores aleatorios en dos columnas
df_131[["garaje","url"]].sample(10)
# Muestreo de valores unicos
df_131["trastero"].unique()
# Muestreo de valores unicos
df_131["ascensor"].unique()
# Muestreo de valores unicos 
df_131["porteria"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10529 entries, 0 to 10528
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               10529 non-null  object 
 1   url                 10529 non-null  object 
 2   precio              10529 non-null  object 
 3   precio_anterior     55 non-null     float64
 4   descuento           55 non-null     object 
 5   EUR/m2              8148 non-null   float64
 6   m2_constr           10501 non-null  float64
 7   distrito            10507 non-null  object 
 8   cod_distrito        10460 non-null  float64
 9   barrio              10117 non-null  object 
 10  cod_barrio          10142 non-null  float64
 11  zona                5724 non-null   object 
 12  calle               1107 non-null   object 
 13  detalle             6014 non-null   object 
 14  consumoce_ano       3438 non-null   object 
 15  letrace             1422 non-null   object 
 16  emis

array(['Portero media jornada', nan, 'Portero todo el día'], dtype=object)

In [388]:
df17['cantidad_visitas'] = np.where((df17['cantidad_visitas'].isnull()) | (df17['cantidad_visitas'] == 0), df17['n.visitas'], df17['cantidad_visitas'])


In [389]:
df17['n.visitas'].value_counts(dropna=False)

n.visitas
NaN       9647
30.00      153
40.00      139
20.00      130
50.00       97
          ... 
170.00       2
140.00       1
200.00       1
190.00       1
180.00       1
Name: count, Length: 21, dtype: int64

In [390]:
# Guardar el DataFrame en un archivo CSV sin incluir el índice
#df.to_csv("dfpisosCompleto17012025.csv", index=False)


In [391]:
df17["porteria"].value_counts(dropna=False)

porteria
NaN                      10372
Portero todo el día        153
Portero media jornada        4
Name: count, dtype: int64

# revision columnas

In [392]:


# Supongamos que df17 es tu DataFrame
#df17['porteria'] = df17['porteria'].replace('0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('0.0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('Climatización', 'climatizacion')
df17['calefaccion'] = df17['calefaccion'].replace('Climatizaciòn', 'climatizacion')
df17['tiene_armario'] = df17['tiene_armario'].replace(0.0, np.nan)
df17.loc[df17['web'].isnull(), 'web'] = 'Indomio.com'
df17['tiene_piscina'] = df17['tiene_piscina'].replace(0, np.nan)


df17['porteria'] = df17['porteria'].replace('1', 'Portero todo el día')


## calefaccion

In [393]:
df17["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                        6949
Central                     927
Individual                  646
1.0                         578
Gas natural                 520
                           ... 
Por climatizacion             1
Contadores individuales       1
Bomba de calor                1
Conducto                      1
Gas natural y biomasa         1
Name: count, Length: 62, dtype: int64

In [394]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto
    
 
    
    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)
    
    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)
      
   
    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)
    
   
    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)    
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else: 
        radiadores.add(categoria)
    

# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Central, suelo radiante', 'Central, por radiadores, de gasóil', 'Central', 'Central, suelo radiante, de gas', 'Central, por radiadores, de gas', 'Comunitaria', 'Comunitario', 'Agua caliente y calefacción central con contador i', 'Central individualizada', 'Calefaccion Central', 'Centralizada', 'Calefaccion comunitaria', 'Central, por radiadores', 'Central, de gasóil', 'Central, de gas', 'Central, alimentación eléctrica', 'Calefacción central individualizada', 'Central, por aire, de gas']
Individual: ['Bomba de calor', 'Individual, suelo radiante, de gas', 'Calefaccion Individual', 'Termo electrico', 'Por climatizacion', 'Bomba frio/calor', 'Contadores individuales', 'Individual, por radiadores', 'Individual, de gas', 'Individual, suelo radiante', 'Bomba frio calor', 'Individual, por radiadores, de gas', 'Individual, de gasóil', 'Individual, por aire, por bomba de calor', 'climatizacion', 'Bomba frio-calor', 'Individual con gas natural', 'Individual, por radiadores, alimentac

In [395]:
df17["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN                     6994
Más de 50 años          1018
Entre 30 y 50 años       205
 Mas de 50 anos          131
1900.0                   118
                        ... 
 Entre 10 y  15 Anos       1
 1949                      1
 1943                      1
 1928                      1
1944.0                     1
Name: count, Length: 188, dtype: int64

In [396]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10295
6.00        45
4.00        42
3.00        32
7.00        25
5.00        24
8.00        21
2.00        14
10.00       14
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

In [397]:
df17.columns = df17.columns.str.strip()

In [398]:
df17.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio',
       ...
       'cantidad_dormitorios', 'orientacion_este', 'orientacion_norte',
       'orientacion_norte,_sur', 'orientacion_oeste', 'orientacion_sur',
       'orientacion_sur,_este', 'sistema_alarma', 'n.visitas', 'grupo_cocina'],
      dtype='object', length=110)

In [399]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10295
6.00        45
4.00        42
3.00        32
7.00        25
5.00        24
8.00        21
2.00        14
10.00       14
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

## aire acondicionado

In [400]:
df17["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN                                                 5976
0                                                   1038
0.0                                                  845
Individual                                           645
Frío                                                 480
                                                    ... 
Aire frio/ calor por conductos/suelo refrescante       1
No disponible                                          1
Centralizada, frío                                     1
Por conductos en toda la casa                          1
Climatizacion por conductos frio y calor               1
Name: count, Length: 36, dtype: int64

In [401]:
df17["aire_acondicionado"].unique()


array([nan, 'Preinstalación', 'Individual, frío/calor',
       'Individual, frío', 'Individual', 'Frío/calor',
       'Centralizada, frío', 'Centralizada, frío/calor',
       'Centralizada, calor', 'Centralizada', 'Frío', 'Frío y calor',
       'Splites repartidos', 'Por conductos en toda la casa',
       'Aerotermia', 'Independiente', 'Con bomba de calor', 'A/a f/c',
       'Suelo refrigerante', 'Preinstalado', 'Frío-calor',
       'Suelo radiante refrescante', 'Sistema air zone',
       'Climatizacion por conductos frio y calor',
       'Frió/calor por conductos', 'Central', 'Suelo radiante',
       'Tiene la instalción hecha en toda la casa por cond',
       'Geotermia y aerotermia', 'En toda la casa, por conductos',
       'Aire frio/ calor por conductos/suelo refrescante',
       'No disponible', '1', '0', '0.0', '1.0'], dtype=object)

In [402]:
# Listas de valores para cada grupo
eficientes_val_columna = [
    'Aerotermia', 'Geotermia y aerotermia', 'Suelo radiante refrescante', 'Suelo refrigerante',
    'Sistema air zone', 'Climatizacion por conductos frio y calor', 'Frió/calor por conductos', 
    'Suelo radiante', 'Aire frio/ calor por conductos/suelo refrescante', 'Por conductos en toda la casa', 
    'Tiene la instalción hecha en toda la casa por cond', 'En toda la casa, por conductos'
]

normales_val_columna = [
    'Individual, frío/calor', 'Individual, frío', 'Individual', 'Frío/calor', 'Centralizada, frío', 
    'Centralizada, frío/calor', 'Centralizada, calor', 'Central', 'Frío', 'Frío y calor', 
    'Splites repartidos', 'Independiente', 'Con bomba de calor', 'A/a f/c', 
    'Aire frio/calor por conductos/suelo refrescante', '1', '1.0'
]

sin_aire_val_columna = ['Preinstalación', 'Preinstalado', 'No disponible', '0', '0.0']

# Función para clasificar los sistemas
def clasificar_sistema(valor):
    if valor in eficientes_val_columna:
        return 'Eficiente'
    elif valor in normales_val_columna:
        return 'Normal'
    elif valor in sin_aire_val_columna:
        return 'Sin Aire'
    else:
        return 'Desconocido'  # Si el valor no coincide con ninguno de los grupos




In [403]:
# Crear una nueva columna en el DataFrame
df17['clasificacion_aire'] = df17['aire_acondicionado'].apply(clasificar_sistema)


In [404]:
df17["jardin"].unique()

array([nan, 'Privado', 'Sin jardín', 'Comunitario', '0.0', '1.0'],
      dtype=object)

In [405]:
# Ejemplo de uso con una sola columna
porcentaje = porcentaje_nulos(df17, 'plantas_edificio')
print(f"Porcentaje de valores nulos en 'plantas_edificio': {porcentaje.get('plantas_edificio', 'Column not found'):.2f}%")


Porcentaje de valores nulos en 'plantas_edificio': 97.78%


In [406]:

# Ejemplo de uso
columnas = ["inmueble_ingresos", "alquiler_opcion_a_compra", "disponibilidad","nuda_propiedad","sup_comercial"]  # Reemplaza con las columnas de tu DataFrame
porcentaje_resultados = porcentaje_nulos(df17, columnas)

# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en inmueble_ingresos: 99.92%
Porcentaje de nulos en alquiler_opcion_a_compra: 99.97%
Porcentaje de nulos en disponibilidad: 95.55%
Porcentaje de nulos en nuda_propiedad: 99.98%
Porcentaje de nulos en sup_comercial: 96.46%


In [407]:
df17["disponibilidad"].value_counts(dropna=False)

disponibilidad
NaN      10060
Libre      469
Name: count, dtype: int64

La columna disponibilidad no aporta nada

In [408]:
df17["alquiler_opcion_a_compra"].value_counts(dropna=False)

alquiler_opcion_a_compra
NaN     10526
1.00        3
Name: count, dtype: int64

In [409]:
df17.query("(inmueble_ingresos == 1) | (alquiler_opcion_a_compra == 1) | (nuda_propiedad == 1)").count()


title                    13
url                      13
precio                   13
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            2
n.visitas                 0
grupo_cocina             13
clasificacion_aire       13
Length: 111, dtype: int64

## sup_comercial

In [410]:
df17[df17['sup_comercial'].notnull()].head()

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina,clasificacion_aire
2,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",https://www.indomio.es/anuncios/81008864/,200000.00,NaN,NaN,500.00,400.00,Arganzuela,2.00,Legazpi,24.00,Legazpi,plaza de italia,Casa plurifamiliar Plaza de Italia,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,Sótano (-5),NaN,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,23.00,NaN,Privado,NaN,NaN,NaN,NaN,401.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio,NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,1.00,NaN,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion,Desconocido
18,"Piso de tres habitaciones buen estado, Ensanche de Vallecas-Valdecarros, Madrid",https://www.indomio.es/anuncios/87841991/,450000.00,NaN,NaN,4.59,98.00,Villa de Vallecas,18.00,EnsanchedeVallecas,183.00,Ensanche de Vallecas,NaN,Piso de tres habitaciones buen estado,desconocido,NaN,No indicado,Piso,NaN,Bueno / Habitable,No,1,3,3.00,2,1.00,NaN,1 en aparcamiento/garaje público,1.00,1.00,NaN,Individual,2010.0,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,203.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,1.00,0.00,0.00,0.00,"Bonito piso en el Ensanche, cerca del metro y todos los sservicios. Cerca del colegio Stella Maris, metro la gavia y cc.la gavia. Bien comunicado con la A3. Buen estado de conservación.",NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,0.00,NaN,183.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion,Desconocido
19,"Piso de dos habitaciones buen estado, San Diego, Madrid",https://www.indomio.es/anuncios/88009171/,210000.00,NaN,NaN,2.80,75.00,Puente de Vallecas,13.00,SanDiego,132.00,San Diego,NaN,Piso de dos

In [411]:
df17['tipo_inmueble'].value_counts(dropna=False)

tipo_inmueble
NaN          9959
económico     276
medio         173
señorial      121
Name: count, dtype: int64

In [412]:
df17['tiene_armario'].value_counts(dropna=False)

tiene_armario
NaN     9811
1.00     718
Name: count, dtype: int64

In [413]:
df17['cancha_tenis'].value_counts(dropna=False)

cancha_tenis
NaN     7091
0.00    3435
1.00       3
Name: count, dtype: int64

In [414]:
df17['chimenea'].value_counts(dropna=False)

chimenea
NaN     6209
0.00    4268
1.00      52
Name: count, dtype: int64

In [415]:
df17[df17['chimenea'] == 1][['calefaccion', 'chimenea']]

,calefaccion,chimenea
22,NaN,1.00
81,NaN,1.00
208,"Central, por radiadores, de gas",1.00
212,"Individual, por radiadores, de gas",1.00
227,NaN,1.00
...,...,...
9581,Individual,1.00
9746,1.0,1.00
9749,1.0,1.00
9750,1.0,1.00


In [416]:
df17['fibra_optica'].value_counts(dropna=False)

fibra_optica
NaN     7091
0.00    3396
1.00      42
Name: count, dtype: int64

In [417]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9425
1.00    1104
Name: count, dtype: int64

In [418]:
df17['carpinteria_exterior_doble_vidrio/pvc'].value_counts(dropna=False)

carpinteria_exterior_doble_vidrio/pvc
NaN     7091
0.00    3419
1.00      19
Name: count, dtype: int64

In [ ]:
df17[df17['tipo_inmueble'].notnull()]

## cocina

In [420]:
df17['cocina'].value_counts(dropna=False)

cocina
NaN                                   8927
Cocina equipada                        590
Cocina independiente                   323
Cocina amueblada                       190
Independiente                          131
                                      ... 
Equipada con excelentes calidades        1
Con entrada independiente                1
Independiente amueblada y equipada       1
Independiente con despensa               1
Amueblada reformada                      1
Name: count, Length: 88, dtype: int64

In [421]:
df17['cocina'].unique()

array([nan, 'Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada', 'Cocina independiente amueblada',
       'Cocina americana', 'Amueblada con electrodomésticos',
       'Cocina equipada', 'Americana', 'Independiente y equipada',
       'Amueblada sin electrodomésticos', 'Cocina amueblada',
       'Independiente', 'Amueblada, equipada', 'Individual',
       'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',
       'Co ofi

In [422]:
df17.query("amueblado=='Sólo cocina amueblada' and cocina.notnull()")[['amueblado','cocina']]

,amueblado,cocina
493,Sólo cocina amueblada,Cocina abierta amueblada
1286,Sólo cocina amueblada,Cocina independiente amueblada
1755,Sólo cocina amueblada,Cocina independiente amueblada
1806,Sólo cocina amueblada,Cocina independiente amueblada
2831,Sólo cocina amueblada,Cocina independiente amueblada
9558,Sólo cocina amueblada,Cocina independiente amueblada


In [423]:
#pongo el valor de cocina_1 en cocina y elimino el campo cocina_1
#df17.loc[df17['cocina_1'].notnull(), 'cocina'] = df17['cocina_1']
#el campo cocina_equipada que aparece en redpiso lo integro en cocina y luebo elimino el campo
#df17.loc[df17['cocina_equipada']==1, 'cocina'] = 'Cocina equipada'
df17.loc[df17['cocina']=='Francesa', 'cocina'] = 'independiente y amueblada'
df17.loc[(df17['amueblado'] == 'Sólo cocina amueblada') & (df17['cocina'].notnull()), 'cocina'] = df17['cocina'] + ' amueblada'
df17.loc[df17['cocina']=='Equipafda americana', 'cocina'] = 'Equipada americana'

#df17.drop(columns=['cocina_1'], inplace=True)
#df17.drop(columns=['cocina_equipada'], inplace=True)


In [424]:
def agrupar_cocinas(tipo):
    categorias = []
    if pd.isnull(tipo):
        categorias.append('sin_informacion')
    elif isinstance(tipo, str):
        if 'abierta' in tipo.lower() or 'abierto' in tipo.lower() or 'americana' in tipo.lower() or 'integrada' in tipo.lower():
            categorias.append('abierta')
        if 'independiente' in tipo.lower():
            categorias.append('independiente')
        if 'amplia' in tipo.lower():
            categorias.append('amplia')
        if 'isla' in tipo.lower():
            categorias.append('con isla')
        if 'amueblada' in tipo.lower():
            categorias.append('amueblada')
        if 'con electrodomesticos' in tipo.lower() or 'equipada' in tipo.lower() or 'con elec.' in tipo.lower():
            categorias.append('equipada')       
        if 'sin amueblar' in tipo.lower():
            categorias.append('no amueblada')
        if 'reformada' in tipo.lower():
            return 'reformada'
        elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
            return 'Sin reforma'
        if not categorias:
            categorias.append('sin categoria')
        return ', '.join(categorias)
    else:
        return 'error tipo campo no texto'
    return ', '.join(categorias)


In [425]:
# Aplica la función de agrupación
df17['grupo_cocina'] = df17['cocina'].apply(agrupar_cocinas)

print(df17[['cocina', 'grupo_cocina']])


               cocina     grupo_cocina
0                 NaN  sin_informacion
1      Cocina abierta          abierta
2                 NaN  sin_informacion
3                 NaN  sin_informacion
4                 NaN  sin_informacion
...               ...              ...
10524             NaN  sin_informacion
10525             NaN  sin_informacion
10526             NaN  sin_informacion
10527             NaN  sin_informacion
10528             NaN  sin_informacion

[10529 rows x 2 columns]


In [426]:
df17.query("cocina.notnull()")['cocina'].unique()

array(['Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada amueblada',
       'Cocina independiente amueblada amueblada', 'Cocina americana',
       'Amueblada con electrodomésticos', 'Cocina equipada', 'Americana',
       'Independiente y equipada', 'Amueblada sin electrodomésticos',
       'Cocina amueblada', 'Independiente', 'Amueblada, equipada',
       'Individual', 'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',

In [427]:
df17.query("amueblado=='Sólo cocina amueblada'").count()

title                    78
url                      78
precio                   78
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            5
n.visitas                 0
grupo_cocina             78
clasificacion_aire       78
Length: 111, dtype: int64

In [428]:
df17['amueblado'].value_counts(dropna=False)

amueblado
NaN                       7606
0                         1469
0.0                        721
Sí                         305
No                         175
1.0                        161
Sólo cocina amueblada       78
Parcialmente amueblado      13
1                            1
Name: count, dtype: int64

In [429]:
## interior, exterior
df17['interior'].value_counts(dropna=False)

interior
0.00    7785
NaN     2651
1.00      93
Name: count, dtype: int64

In [430]:

df17['exterior'].value_counts(dropna=False)

exterior
0.00    7479
1.00    1815
NaN     1235
Name: count, dtype: int64

In [431]:
df17["vista_zona"].unique()

array([nan, 'Exterior'], dtype=object)

In [432]:
# Si el valor de la columna 'exterior' es nulo y la columna 'vista_zona' tiene el valor 'Exterior', poner en la columna 'exterior' un 1
df17.loc[df17['exterior'].isnull() & (df17['vista_zona'] == 'Exterior'), 'exterior'] = 1

# Eliminar la columna 'vista_zona'
df17.drop(columns=['vista_zona'], inplace=True)

In [433]:

df17['interior_y_exterior'].value_counts(dropna=False)

interior_y_exterior
NaN     7091
0.00    3429
1.00       9
Name: count, dtype: int64

In [434]:

df17.query("interior.isnull() & exterior.isnull() & interior_y_exterior.isnull()")["web"].unique()

array(['redpiso', 'pisosmadrid'], dtype=object)

## piscina

In [435]:


df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9425
1.00    1104
Name: count, dtype: int64

In [436]:
df17['piscina'].value_counts(dropna=False)

piscina
 NaN                                          4811
 -                                            3150
0.00                                          1896
 0                                             310
1.00                                           142
 Comunitaria                                   139
 Propia                                         75
 1                                               4
 Con salorium                                    1
 1 piscina de adultos y 1 piscina de niños       1
Name: count, dtype: int64

In [437]:

#asignamos el valor tiene_piscina sin indicar el tipo
df17.loc[(df17['piscina'] != 0.0) & (df17['piscina'] != 0) & (df17['piscina'] != '-') & (df17['piscina'].notnull()), 'tiene_piscina'] = 1

In [438]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9425
1.00    1104
Name: count, dtype: int64

In [439]:

df17.query("tiene_piscina.notnull()")["web"].unique()

array(['Indomio.com', 'pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

## porton electrico, puerta blindada

In [440]:

## portn_electrico,puerta_blindad

columnas = ["porton_electrico", "puerta_blindada", "alarma","videoportero","lujosa","descripcion_extendida"]  
porcentaje_resultados = porcentaje_nulos(df17, columnas)


# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en porton_electrico: 67.35%
Porcentaje de nulos en puerta_blindada: 67.35%
Porcentaje de nulos en alarma: 47.74%
Porcentaje de nulos en videoportero: 58.97%
Porcentaje de nulos en lujosa: 67.35%
Porcentaje de nulos en descripcion_extendida: 45.42%


In [441]:

df17['porton_electrico'].value_counts(dropna=False)

porton_electrico
NaN     7091
0.00    3421
1.00      17
Name: count, dtype: int64

In [442]:

df17['puerta_blindada'].value_counts(dropna=False)

puerta_blindada
NaN     7091
0.00    3337
1.00     101
Name: count, dtype: int64

In [443]:

df17['alarma'].value_counts(dropna=False)

alarma
0.00    5409
NaN     5027
1.00      93
Name: count, dtype: int64

In [444]:

df17.query("alarma==1 | puerta_blindada==1")[['precio','alarma','puerta_blindada']]

,precio,alarma,puerta_blindada
0,2000000.00,0.00,1.00
1,287000.00,0.00,1.00
18,450000.00,0.00,1.00
52,199990.00,0.00,1.00
79,176000.00,0.00,1.00
...,...,...,...
10422,1939000,1.00,NaN
10469,800000,1.00,NaN
10508,870000,1.00,NaN
10509,515000,1.00,NaN


In [445]:

df17['videoportero'].value_counts(dropna=False)

videoportero
NaN     6209
0.00    3969
1.00     351
Name: count, dtype: int64

In [446]:
## lujosa

In [447]:

df17['lujosa'].value_counts(dropna=False)

lujosa
NaN     7091
0.00    1872
1.00    1566
Name: count, dtype: int64

In [448]:

df17.query("videoportero==1").count()

title                    351
url                      351
precio                   351
precio_anterior            0
descuento                  0
                        ... 
orientacion_sur,_este      0
sistema_alarma             1
n.visitas                282
grupo_cocina             351
clasificacion_aire       351
Length: 110, dtype: int64

## descripcion_extendida

################ REVISAR #################

€_comunidad_mes

In [449]:
df17['€_comunidad_mes'].value_counts(dropna=False)

€_comunidad_mes
NaN                 9191
Más de 100 €         350
Entre 40 y 60 €      100
Entre 80 y 100 €      95
Entre 60 y 80 €       80
                    ... 
336                    1
175                    1
325                    1
525                    1
60 €.-                 1
Name: count, Length: 314, dtype: int64

In [450]:
df17['gastos_comunidad'].value_counts(dropna=False)

gastos_comunidad
NaN               10436
Comunidad 5000        9
€ 50/mes              6
€ 100/mes             5
Comunidad 4500        4
                  ...  
€ 138/mes             1
€ 90/mes              1
€ 120/mes             1
€ 61/mes              1
€ 140/mes             1
Name: count, Length: 58, dtype: int64

In [451]:
df17.shape

(10529, 110)

In [452]:
df17.query("`€_comunidad_mes`.notnull() | gastos_comunidad.notnull()")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
13,Indomio.com,NaN,60
...,...,...,...
9616,Indomio.com,€ 100/mes,NaN
9620,Indomio.com,€ 100/mes,NaN
9621,Indomio.com,€ 140/mes,NaN
9623,Indomio.com,€ 50/mes,NaN


In [453]:
df17.query("`€_comunidad_mes`.notnull() and  gastos_comunidad.isnull() ")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
13,Indomio.com,NaN,60
...,...,...,...
7866,pisos.com,NaN,Más de 100 €
7873,pisos.com,NaN,Más de 100 €
7874,pisos.com,NaN,38
7876,pisos.com,NaN,Entre 20 y 40 €


In [454]:
df17.query(" gastos_comunidad.notnull() and `€_comunidad_mes`.isnull() ")[['web','url','gastos_comunidad','€_comunidad_mes']]

,web,url,gastos_comunidad,€_comunidad_mes
7892,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-los-rosales-villaverde-madrid-madrid-RP2372023110992,Comunidad 7400,NaN
7914,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-villaverde-madrid-madrid-RP2372024127521,Comunidad 4000,NaN
7976,redpiso,https://www.redpiso.es/inmueble/atico-en-venta-en-calle-san-clodoaldo-ventas-ciudad-lineal-madrid-madrid-RP162024125933,Comunidad 9900,NaN
8030,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-godella-san-cristobal-villaverde-madrid-madrid-RP2842024125361,Comunidad 5000,NaN
8037,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-almendrales-almendrales-usera-madrid-madrid-RP392024129914,Comunidad 2000,NaN
...,...,...,...,...
9616,Indomio.com,https://www.indomio.es/anuncios/96986027/,€ 100/mes,NaN
9620,Indomio.com,https://www.indomio.es/anuncios/96967621/,€ 100/mes,NaN
9621,Indomio.com,https://www.indomio.es/anuncios/96979909/,€ 140/mes,NaN
9623,Indomio.com,https://www.indomio.es/anuncios/96696025/,€ 50/mes,NaN


In [455]:
#En redpiso los valores hay que dividirlos entre 100 al procesarlo se quito la coma que tenia en 20,00 ahora aparece con 2000
#pisos madrid y no tiene gastos de comunidad

In [456]:
df17.gastos_comunidad.unique()

array([nan, 'Comunidad 7400', 'Comunidad 4000', 'Comunidad 9900',
       'Comunidad 5000', 'Comunidad 2000', 'Comunidad 4500',
       'Comunidad 18000', 'Comunidad 10000', 'Comunidad 11000',
       'Comunidad 6500', 'Comunidad 3500', 'Comunidad 15000',
       'Comunidad 6900', 'Comunidad 3000', 'Comunidad 3750',
       'Comunidad 23600', 'Comunidad 6000', 'Comunidad 38000',
       '€ 175/mes', '€ 50/mes', '€ 37/mes', '€ 35/mes', '€ 112/mes',
       '€ 457/mes', '€ 95/mes', '€ 40/mes', '€ 111/mes', '€ 148/mes',
       '€ 160/mes', '€ 80/mes', '€ 138/mes', '€ 90/mes', '€ 120/mes',
       '€ 61/mes', '€ 451/mes', '€ 350/mes', '€ 43/mes', '€ 230/mes',
       '€ 125/mes', '€ 55/mes', '€ 100/mes', '€ 72/mes', '€ 58/mes',
       '€ 283/mes', '€ 27/mes', '€ 33/mes', '€ 26/mes', '€ 103/mes',
       '€ 163/mes', '€ 60/mes', '€ 30/mes', '€ 180/mes', '€ 280/mes',
       '€ 404/mes', '€ 109/mes', '€ 45/mes', '€ 140/mes'], dtype=object)

In [457]:
# Sustituir las palabras 'Comunidad', '€', y '/mes' por vacío en 'gastos_comunidad'
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('Comunidad', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('€', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('/mes', '')
# Eliminar espacios en blanco adicionales
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.strip()
 #Convertir solo los valores no nulos de 'gastos_comunidad' a tipo float
df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'] = df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'].astype(float)
# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
df17.loc[(df17['web'] == 'redpiso') & (df17['gastos_comunidad'].notnull()), 'gastos_comunidad'] /= 100

# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
# Convertir la columna 'gastos_comunidad' a tipo float
#df17['gastos_comunidad'] = pd.to_numeric(df17['gastos_comunidad'], errors='coerce')
#df17.loc[df17['web'] == 'redpiso', 'gastos_comunidad'] /= 100

In [458]:
df17.gastos_comunidad.unique()

array([nan, 74.0, 40.0, 99.0, 50.0, 20.0, 45.0, 180.0, 100.0, 110.0, 65.0,
       35.0, 150.0, 69.0, 30.0, 37.5, 236.0, 60.0, 380.0, 175.0, 37.0,
       112.0, 457.0, 95.0, 111.0, 148.0, 160.0, 80.0, 138.0, 90.0, 120.0,
       61.0, 451.0, 350.0, 43.0, 230.0, 125.0, 55.0, 72.0, 58.0, 283.0,
       27.0, 33.0, 26.0, 103.0, 163.0, 280.0, 404.0, 109.0, 140.0],
      dtype=object)

In [459]:
# Obtener los valores únicos de la columna '€_comunidad_mes'
df17['€_comunidad_mes'].unique()


array(['100', '123', nan, '55', '60', '258', '50', '92', '27', '90',
       '132', '128', '113', '800', '180', '1.300', '65', '625', '34',
       '150', '308', '146', 'Ningún gasto de comunidad', '310', '350',
       '110', '96', '120', '210', '70', '1.000', '250', '84', '247', '3',
       '56', '450', '40', '200', '75', '190', '166', '15', '45', '228',
       '72', '9', '294', '30', '160', '430', '95', '104', '63', '13',
       '85', '119', '300', '168', '340', '99', '164', '130', '241', '167',
       '5', '33', '111', '420', '206', '500', '10', '14', '540', '750',
       '148', '147', '386', '17', '76', '230', '205', '46', '47', '400',
       '32', '12', '36', '82', '155', '303', '207', '486', '83', '80',
       '66', '98', '178', '140', '52', '64', '54', '42', '29', '88', '61',
       '1.600', '440', '170', '57', '105', '142', '255', '125', '600',
       '324', '67', '269', '285', '38', '39', '102', '270', '44', '103',
       '617', '552', '311', '457', '192', '327', '288', '316', '

In [460]:
# Usar comillas invertidas para nombres de columnas con caracteres especiales
df17.query("`€_comunidad_mes` == 'Comunidad: 320€ mensual  / ibi 1.040,96€ anual'")[["web"]]

,web
4865,pisos.com


#############revisar

#Tengo que ir al origen para separar los valores de gastos de comunidad y de ibi

In [461]:
# Reemplazar valores nulos en 'gastos_comunidad' con los valores de '€_comunidad_mes'
df17['gastos_comunidad'] = df17['gastos_comunidad'].fillna(df17['€_comunidad_mes'])
# Eliminar la columna '€_comunidad_mes'
df17.drop(columns=['€_comunidad_mes'], inplace=True)

In [462]:

# Reemplazar 'No tiene' y 'Ningún gasto de comunidad' por '0'
df17['gastos_comunidad'] = df17['gastos_comunidad'].replace({'No tiene': '0', 'Ningún gasto de comunidad': '0'})


In [463]:
df17['acceso_minusvalido'].value_counts(dropna=False)

acceso_minusvalido
 NaN    8794
 0      1146
0.00     498
1.00      69
 1        10
 Sí        7
 No        5
Name: count, dtype: int64

## acceso_minusvalidos

In [464]:
# Reemplazar 'Sí' por 1 y 'No' por 0 en la columna 'acceso_minusvalido'
df17['acceso_minusvalido'] = df17['acceso_minusvalido'].replace({'Sí': 1, 'No': 0})

# Renombrar la columna 'acceso_minusvalido' a 'acceso_discapacidad'
df17.rename(columns={'acceso_minusvalido': 'acceso_discapacidad'}, inplace=True)

## orientacion

In [465]:
df17["orientacion"].value_counts()

orientacion
Sur                  312
Este                 263
Orientacion Sur      162
Orientacion Este     154
Oeste                148
                    ... 
Este-oeste             1
Norte y este           1
Oeste y este           1
Sur, oeste             1
Sur, este y norte      1
Name: count, Length: 40, dtype: int64

In [466]:
# Eliminar 'Orientacion' y limpiar espacios adicionales
df17['orientacion'] = df17['orientacion'].str.replace('Orientacion', '').str.strip()

# Sustituir ',' y '-' por '_' y poner los valores en minúsculas en la columna 'orientacion'
df17['orientacion'] = df17['orientacion'].str.replace(',', '_').str.replace('/', '_').str.replace('|', '_').str.replace('y', '_').str.replace('-', '_').str.lower()

# Eliminar espacios adicionales entre palabras
df17['orientacion'] = df17['orientacion'].str.replace(r'\s+', '_', regex=True).str.strip()
df17['orientacion'] = df17['orientacion'].str.replace('__', '_')

# Reemplazar múltiples valores en la columna 'orientacion' por np.nan
df17['orientacion'] = df17['orientacion'].replace({
    '__':'_',
    '_1': np.nan,
    'medio_día': np.nan,
    'sur_en_la_terraza__dormitorios': np.nan,
    'vistas_al_palacio_de_parcent.': np.nan,
    'varias':np.nan
})

df17['orientacion'] = df17['orientacion'].str.replace('__', '_')



In [467]:
# Obtener los valores únicos de la columna 'orientacion' donde los valores no son nulos
df17.loc[df17['orientacion'].notnull(), 'orientacion'].unique()


array(['norte', 'sur_este', 'este', 'oeste', 'sur', 'norte_sur',
       'noroeste', 'sureste', 'suroeste', 'noreste', 'nordeste',
       'a_la_montaña', 'este_oeste', 'sur_este_norte',
       'este_noreste_suroeste', 'sur_oeste', 'oeste_este', 'norte_este',
       'este_oeste_norte', 'sudeste', 'sur_norte'], dtype=object)

In [468]:


# Procesar la columna 'orientacion'
for index, value in df17["orientacion"].items():
    # hasta los nan los convierte a string, recordar luego cambiar 'nan' por nan de nuevo
    value_str = str(value)
    if re.search(r'\bnorte\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
    if re.search(r'\bsur\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
    if re.search(r'\beste\b', value_str):
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\boeste\b', value_str):
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bnoroeste\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
        df17.at[index, "orientacion_oeste"] = 1

# Revisar las columnas orientacion_norte,_sur y orientacion_sur,_este
# y asegurarnos de que las combinaciones sean coherentes
if "orientacion_norte,_sur" in df17.columns:
    for index, value in df17["orientacion_norte,_sur"].items():
        if value == 1:
            df17.at[index, "orientacion_norte"] = 1
            df17.at[index, "orientacion_sur"] = 1

if "orientacion_sur,_este" in df17.columns:
    for index, value in df17["orientacion_sur,_este"].items():
        if value == 1:
            df17.at[index, "orientacion_sur"] = 1
            df17.at[index, "orientacion_este"] = 1


In [469]:
# Filtrar filas donde todas las columnas relevantes no sean nulas
filtered_df = df17[
    df17["orientacion"].notnull() |
    df17["orientacion_norte"].notnull() |
    df17["orientacion_sur"].notnull() |
    df17["orientacion_este"].notnull() |
    df17["orientacion_oeste"].notnull() |
    df17["orientacion_norte,_sur"].notnull() |
    df17["orientacion_sur,_este"].notnull()
]

# Mostrar las columnas deseadas
result = filtered_df[["orientacion", "orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]]
print(result)


     orientacion  orientacion_norte  orientacion_sur  orientacion_este  \
94         norte               1.00              NaN               NaN   
175     sur_este                NaN              NaN               NaN   
229         este                NaN              NaN              1.00   
311     sur_este                NaN              NaN               NaN   
362        oeste                NaN              NaN               NaN   
...          ...                ...              ...               ...   
9631         NaN                NaN             0.00              0.00   
9632         NaN                NaN             0.00              0.00   
9633         NaN                NaN             0.00              0.00   
9634         NaN                NaN             0.00              0.00   
9635         NaN                NaN             0.00              0.00   

      orientacion_oeste  
94                  NaN  
175                 NaN  
229                 NaN  
311    

In [470]:
#eliminacion de las columnas orientacion,orientacion_sur,_este,orientacion_norte,_sur
df17.drop(columns=['orientacion'], inplace=True)
df17.drop(columns=['orientacion_sur,_este'], inplace=True)
df17.drop(columns=['orientacion_norte,_sur'], inplace=True)

,web,url,orientacion,orientacion_este


In [472]:
df17.query("orientacion_norte!=1 and orientacion_sur!=1  and orientacion_este!=1  and orientacion_oeste!=1 ").count()

title                    9016
url                      9016
precio                   9016
precio_anterior            55
descuento                  55
                         ... 
orientacion_sur,_este       0
sistema_alarma            284
n.visitas                 882
grupo_cocina             9016
clasificacion_aire       9016
Length: 109, dtype: int64

In [ ]:
# Poner NaN en las columnas si no cumplen con la condición
df17.loc[df17.query("orientacion_norte!=1 and orientacion_sur!=1 and orientacion_este!=1 and orientacion_oeste!=1").index, ["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = np.nan

# Asegurarse de que si alguna columna tiene valor 1, las demás sean 0
df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]].fillna(0)

## web

In [473]:
df17["web"].value_counts(dropna=False)

web
pisos.com      4728
Indomio.com    3438
redpiso        1470
pisosmadrid     893
Name: count, dtype: int64

In [476]:
df17["fecha_descarga"].value_counts(dropna=False)

fecha_descarga
2024-11-20    3150
2024-10-19    2951
2024-08-19    1777
31/10/2024    1470
24/10/2024     893
06/01/2025     288
Name: count, dtype: int64

In [475]:
# Sustituir NaN en la columna 'fecha_descarga' por '6 de enero de 2025'
df17["fecha_descarga"].fillna("06/01/2025", inplace=True)


C:\Users\extas\AppData\Local\Temp\ipykernel_20600\2424142163.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df17["fecha_descarga"].fillna("06/01/2025", inplace=True)


## certificado_energetico

In [478]:
df17["certificado_energetico"].value_counts(dropna=False)

certificado_energetico
NaN                       4289
No indicado               2201
Disponible                2018
En trámite                1492
Pendiente de completar     526
Exento                       3
Name: count, dtype: int64

In [481]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [485]:
df17.query("consumoce_ano.notnull() | letrace.notnull() | emisiones_co2.notnull()" )[ ["certificado_energetico",'consumoce_ano','letrace','emisiones_co2']]

,certificado_energetico,consumoce_ano,letrace,emisiones_co2
0,Disponible,"≥ 3,51",F,D
1,No indicado,desconocido,NaN,No indicado
2,No indicado,desconocido,NaN,No indicado
3,No indicado,desconocido,NaN,No indicado
4,Disponible,desconocido,E,No indicado
...,...,...,...,...
9631,No indicado,desconocido,NaN,No indicado
9632,No indicado,desconocido,NaN,No indicado
9633,No indicado,desconocido,NaN,Inclasificable
9634,No indicado,desconocido,NaN,No indicado


In [486]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
NaN                   6830
No indicado           2205
Inclasificable         344
E                      140
D                      110
                      ... 
8 kg CO₂/m² añoB         1
79 kg CO₂/m² añoF        1
45 kg CO₂/m² añoG        1
118 kg CO₂/m² añoE       1
83 kg CO₂/m² añoG        1
Name: count, Length: 325, dtype: int64

In [ ]:
df17["consumoce_ano"].unique()

In [ ]:
df17.query("consumoce_ano=='≥ 3,51'")

In [489]:
df17["letrace"].value_counts(dropna=False)

letrace
NaN    9107
E       614
D       269
F       183
G       148
C       116
A        52
B        40
Name: count, dtype: int64

#se encuentra un piso que han modificado los valores en la web
# debe tener https://www.indomio.es/anuncios/97003125/, Consumo de energia:57.9 kWh/m² año, letrace:B, e Emisiones de CO₂:11.08 kg CO₂/m² año B

In [20]:
# Asignar valores específicos si la URL coincide
for index, value in df17["url"].items():
    if value == "https://www.indomio.es/anuncios/97003125/":
        df17.at[index, "consumoce_ano"] = 57.9
        df17.at[index, "letra_ce"] = "B"
        df17.at[index, "emisiones_co2"] = "11.08 kg CO₂/m² año B"


revisar el codigo que trae el fichero scrapingindomio.csv , C:\Mas\Dataset_Proyecto\Ficheros_Ventas\Indomio del 12/12/2024 quiza estamos mirando solo mayusculas

#

In [25]:
df17.query("consumoce_ano=='≥ 3,51'")[['url','letra_ce','emisiones_co2','fecha_descarga']]

,url,letra_ce,emisiones_co2,fecha_descarga
0,https://www.indomio.es/anuncios/75404850/,NaN,D,2024-11-20
11,https://www.indomio.es/anuncios/83875795/,NaN,Pendiente,2024-11-20
12,https://www.indomio.es/anuncios/84201619/,NaN,Exento,2024-11-20
13,https://www.indomio.es/anuncios/84201619/,NaN,Exento,2024-11-20
88,https://www.indomio.es/anuncios/92109663/,NaN,Pendiente,2024-11-20
197,https://www.indomio.es/anuncios/93908759/,NaN,Pendiente,2024-11-20
198,https://www.indomio.es/anuncios/93908771/,NaN,Pendiente,2024-11-20
303,https://www.indomio.es/anuncios/94711699/,NaN,Pendiente,2024-11-20
314,https://www.indomio.es/anuncios/94729483/,NaN,E,2024-11-20
352,https://www.indomio.es/anuncios/94950781/,NaN,D,2024-11-20


CONSUMO Y LETRA
A: Menos de 50 kWh/m²/año
B: 51-90 kWh/m²/año
C: 91-150 kWh/m²/año
D: 151-230 kWh/m²/año
E: 231-330 kWh/m²/año
F: 331-450 kWh/m²/año
G: Más de 450 kWh/m²/año

En el caso de las emisiones:
A: Menos de 10 kg CO2/m²/año
B: 10-20 kg CO2/m²/año
C: 20-35 kg CO2/m²/año
D: 35-55 kg CO2/m²/año
E: 55-75 kg CO2/m²/año
F: 75-100 kg CO2/m²/año
G: Más de 100 kg CO2/m²/año

In [93]:
df17.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio',
       ...
       'orientacion_este', 'orientacion_norte', 'orientacion_norte,_sur',
       'orientacion_oeste', 'orientacion_sur', 'orientacion_sur,_este',
       'sistema_alarma', 'n.visitas', 'grupo_cocina', 'clasificacion_aire'],
      dtype='object', length=109)

In [ ]:
#renombrar las columnas que hay de orientacion para integranlas con orientacion y sus valores

In [366]:
#Revisar que filas no tiene descripcion_extendida , deberian tenerlas todas
df17.query("descripcion_extendida.isnull()")['web'].unique()

array(['Indomio.com', 'pisos.com', 'redpiso'], dtype=object)

In [367]:

df17.query("descripcion_extendida.notnull()")['web'].unique()

array(['Indomio.com', 'redpiso', 'pisosmadrid'], dtype=object)

In [ ]:






# Contar las filas donde 'dormitorios' es igual a 'cantidad_dormitorios'
conteo_filas_iguales = (df17['dormitorios'] == df17['cantidad_dormitorios']).sum()

print(conteo_filas_iguales)



In [42]:
df17.to_csv("dfpisosCompleto17012025.csv",index='False')

In [32]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [46]:
# Contar las filas donde 'dormitorios' es igual a 'cantidad_dormitorios'
conteo_filas_iguales = (df17['dormitorios'] == df17['cantidad_dormitorios']).sum()

print(conteo_filas_iguales)


22


In [29]:
df17["cod_barrio"].value_conts(dropna=False)

AttributeError: 'Series' object has no attribute 'value_conts'